In [ ]:
%%capture
!pip install marqo
!pip install plotly

In [ ]:
from marqo import Client
from marqo.errors import MarqoApiError
mq = Client()
index_name = "stable-diffusion-open-prompts"
index = mq.index(index_name)
print(index.get_stats())

In [ ]:
from PIL import Image
import requests

In [ ]:
from IPython.display import display

In [ ]:
def convert_docker_url_to_local(docker_url):
    return 'http://localhost:8222/' + docker_url.split("http://host.docker.internal:8222/")[1]

In [ ]:
def search_and_display(text, limit=3, verbose=True):
    results = index.search(text, limit=limit)
    # print(results)
    for i, result in enumerate(results["hits"]):
        pic = result["local_image_location"]
        prompt = result["prompt"]
        if pic.startswith("http://host.docker.internal:8222/"):
            #This still only gets the image from local
            # print(pic)
            pic_url = convert_docker_url_to_local(pic)
            # print(pic_url)
            image = Image.open(requests.get(pic_url, stream=True).raw)
            if i == 0:
                search_image = pic
            if verbose:
                print(prompt)
                display(image)
                print()
    return results

In [ ]:
results = search_and_display("cellular automata")

In [ ]:
def whisper_down_the_lane_text_image_text(original_text, limit=3, n_iterations=3, verbose=True):
    prompt = original_text
    image = None
    _limit = limit+1
    query_local_image_paths = []
    for i in range(n_iterations):
        iter_dict = {}
        if image:
            iter_dict["text_query"]= {
                "image" : convert_docker_url_to_local(image),
                "prompt": prompt
            }
        else:
            iter_dict["text_query"] = {
                "image" : image,
                "prompt": prompt
            }
        print(f"iteration {i}: {prompt}")
        text_search_results = search_and_display(prompt, limit=_limit, verbose=verbose)
        top_result = text_search_results["hits"][1]
        image = top_result["local_image_location"]
        prompt = top_result["prompt"]
        iter_dict["image_query"]= {
            "image" : convert_docker_url_to_local(image),
            "prompt": prompt
        }
        print(f"iteration {i}: {image}")
        image_search_results = search_and_display(image, limit=_limit, verbose=verbose)
        top_result = image_search_results["hits"][1]
        image = top_result["local_image_location"]
        prompt = top_result["prompt"]
        query_local_image_paths.append(iter_dict)
    query_local_image_paths.append({"result": {"image":convert_docker_url_to_local(image), "prompt": prompt}})
    return query_local_image_paths

In [ ]:
generative_whispers = whisper_down_the_lane_text_image_text("autonomous vehicle", limit=3, n_iterations=3, verbose=False)

In [ ]:
base_prompt = generative_whispers[0]['text_query']["prompt"]

In [ ]:
image_urls = []
prompts = []
for iteration in generative_whispers:
    for key in iteration.keys():
        q = iteration[key]
        image_url = q["image"]
        prompt = q["prompt"]
        if image_url not in image_urls and image_url:
            prompts.append(prompt)
            image_urls.append(image_url)

In [ ]:
image_urls

In [ ]:
prompts

In [ ]:
%%capture
!git clone https://github.com/volotat/DiffMorph
# %cd DiffMorph
!pip install -r DiffMorph/requirements.txt

In [ ]:
import os

In [ ]:
import shutil

In [ ]:
import unicodedata
import re

def slugify(value, allow_unicode=False):
    """
    Taken from https://github.com/django/django/blob/master/django/utils/text.py
    Convert to ASCII if 'allow_unicode' is False. Convert spaces or repeated
    dashes to single dashes. Remove characters that aren't alphanumerics,
    underscores, or hyphens. Convert to lowercase. Also strip leading and
    trailing whitespace, dashes, and underscores.
    """
    value = str(value)
    if allow_unicode:
        value = unicodedata.normalize('NFKC', value)
    else:
        value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
    value = re.sub(r'[^\w\s-]', '', value.lower())
    return re.sub(r'[-\s]+', '-', value).strip('-_')

In [ ]:
input_dir = os.path.join(".", "input")
output_dir = os.path.join(".", "output")
morph_temp_dir = os.path.join(".", "morph")

In [ ]:
if not os.path.exists(input_dir):
    os.makedirs(input_dir)

In [ ]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
if not os.path.exists(morph_temp_dir):
    os.makedirs(morph_temp_dir)

In [ ]:
from datetime import datetime

now = datetime.now()
date_time = now.strftime("%m-%d-%Y-%H-%M-%S")

In [ ]:
slug = slugify(base_prompt)
if not slug:
    slug = date_time

In [ ]:
input_prompt_folder = os.path.join(input_dir, slug)

In [ ]:
if not os.path.exists(input_prompt_folder):
    os.makedirs(input_prompt_folder)

In [ ]:
output_prompt_folder = os.path.join(output_dir, slug)

In [ ]:
if not os.path.exists(output_prompt_folder):
    os.makedirs(output_prompt_folder)

In [ ]:
def morph_generative_whispers(image_urls):
    import sys
    for i, image_url in enumerate(image_urls):
        src_url = image_url.split('http://localhost:8222/')[1]
        dest_url = os.path.join(input_prompt_folder, f"{i+1}.png")
        shutil.copy(src_url, dest_url)
    #this will create a preds.npy file in the cwd! the rest should organize properly
    image_list = [file for file in os.listdir(input_prompt_folder) if not file.startswith('.')]
    image_list.sort()
    epochs = 10 #@param {type:"number"}
    warp_scale = 0.05 #@param {type:"number"}
    mult_scale = 0.4 #@param {type:"number"}
    add_scale = 0.4 #@param {type:"number"}
    add_first = False #@param {type:"boolean"}
    filenames = []
    for i in range(len(image_list) - 1):
        start = f'{input_prompt_folder}/{image_list[i]}'
        end = f'{input_prompt_folder}/{image_list[i+1]}'
        if add_first:
            !python DiffMorph/morph.py -s $start -t $end -e $epochs -a $add_scale -m $mult_scale -w $warp_scale --add_first
        else:
            !python DiffMorph/morph.py -s $start -t $end -e $epochs -a $add_scale -m $mult_scale -w $warp_scale
        filename = f'{output_prompt_folder}/morph{i:03d}.mp4'
        filenames.append(filename)
        !mv ./morph/morph.mp4 $filename
    #@title Step 4: Join videos together
    with open(os.path.join(input_prompt_folder, 'filenames.txt'), 'w') as f:
        f.write('\n'.join([f"file '{filename}'" for filename in filenames]))
    !ffmpeg -f concat -i filenames.txt -codec copy $output_prompt_folder/final.mp4

In [ ]:
morph_generative_whispers(image_urls)

In [ ]:
y